In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from igraph import Graph

In [3]:
N = 1000
pi=0.1

# Problem 1: Generate instances of RG

In [39]:
def havel_hakimi(sequence):
    # per controllare se una sequenza di degree può dare un grafo
    # inutile in quanto faccio la sequenza in modo che lo sia
    sequence = sorted(sequence, reverse=True)
    while sequence:
        n = sequence.pop(0)
        if n > len(sequence):
            return False
        
        for i in range(n):
            sequence[i] -= 1
            if sequence[i] < 0:
                return False
    
        sequence = sorted([x for x in sequence if x > 0], reverse=True)
    
    return True


def has_duplicates(arr):

    return len(arr) != len(set(arr))


def has_loops(arr):
    for elm in arr:
        if elm[0] == elm[1]:
            return True
        
    return False

In [36]:
def sample_pseudo(N,pi):
    # sampla pseudo grafi con loops e repeated edges
    # distribuzione davvero uniforme
    N1 = int((1-pi)*N)
    if N1%2!=0:
        N1+=1
    N4 = N-N1
    Ntot = N1+4*N4
    d = np.zeros(Ntot,dtype=np.int64)
    for i in range(N1):
        d[i]=i
    for j in range(N1,Ntot):
        d[j] = (j-N1)//4+N1
    d = np.random.permutation(d)
    edgeList = d.reshape((Ntot//2,2))
    return edgeList

def sample_bias1(N, pi):
    # sampla dalla distribuzione biased ma il modo in cui samplo dalla lista è poco ottimale
    N1 = int((1 - pi) * N)
    if N1 % 2 != 0:
        N1 += 1
    N4 = N - N1
    Ntot = N1 + 4 * N4
    M = int(Ntot / 2)
    d = np.concatenate((np.ones(N1), 4 * np.ones(N4)))
    dhat = d.copy()
    edgeList = np.zeros((M, 2))
    i = 0
    while np.sum(dhat) > 0 and i < M:  
        drawn = np.random.choice(N, 2, replace=False, p=dhat / np.sum(dhat))
        if drawn[0] != drawn[1] and not np.any(np.all(edgeList[:i] == drawn, axis=1)):
            edgeList[i, :] = drawn
            i += 1
            dhat[drawn[0]] -= 1
            dhat[drawn[1]] -= 1

    edgeList = edgeList[:i]
    return edgeList    

def sample_mask_list(N,pi):
    # sampla ladist. biased usa aggiorno una mask e appendo una lista
    N1 = int((1-pi)*N)
    if N1%2!=0:
        N1+=1
    N4 = N-N1
    Ntot = N1+4*N4
    d = np.zeros(Ntot,dtype=np.int64)
    for i in range(N1):
        d[i]=i
    for j in range(N1,Ntot):
        d[j] = (j-N1)//4+N1
    mask = np.ones_like(d)
    edgeList=[]
    while np.sum(mask)>0:
        drawn = tuple(sorted(tuple(np.random.choice(d[mask==1],2,replace=False))))
        if drawn[0] != drawn[1] and drawn not in edgeList:
            edgeList.append(drawn) 
            for elm in drawn:
                if elm<N1:
                    mask[elm]=0
                else:
                    ip = N1+(elm-N1)*4
                    s = np.sum(mask[ip:ip+4])
                    mask[ip+4-s] = 0 
    return edgeList
    
def sample_mask_array(N,pi):
    # sampla ladist. biased usa aggiorno una mask e modifico un array
    N1 = int((1-pi)*N)
    if N1%2!=0:
        N1+=1
    N4 = N-N1
    Ntot = N1+4*N4
    M = int(Ntot / 2)
    d = np.zeros(Ntot,dtype=np.int64)
    for i in range(N1):
        d[i]=i
    for j in range(N1,Ntot):
        d[j] = (j-N1)//4+N1
    mask = np.ones_like(d)
    edgeList = np.zeros((M, 2))
    i = 0
    while np.sum(mask)>0 and i<M:
        drawn = np.random.choice(d[mask==1],2,replace=False)
        if drawn[0] != drawn[1] and not np.any(np.all(edgeList[:i] == drawn, axis=1)):
            edgeList[i, :] = drawn
            i += 1
            for elm in drawn:
                if elm<N1:
                    mask[elm]=0
                else:
                    ip = N1+(elm-N1)*4
                    s = np.sum(mask[ip:ip+4])
                    mask[ip+4-s] = 0 
    edgeList = edgeList[:i]
    return edgeList


def sample_igraph(N,pi):
    # come comparison, non sampla sempre grafi adeguati
    N1 = int((1-pi)*N)
    if N1%2!=0:
        N1+=1
    N4 = N-N1
    d = np.concatenate((np.ones(N1,dtype=int), 4 * np.ones(N4,dtype=int))) 
    #print(f"the sequence is graphical: {havel_hakimi(d)}")
    g = Graph.Degree_Sequence(d, method="simple") 
    return g.get_edgelist()


def sample_from_pseudo(N,pi):
    g = sample_pseudo(N,pi):
    

In [71]:
count=0
n=1000
for i in range(n):
    g=sample_igraph(N,pi+0.2)
    if has_duplicates(g) or has_loops(g):
        count+=1
count/n

0.852

In [51]:
count=0
for i in range(100):
    g=sample_mask_list(N,pi)
    if has_duplicates(g) or has_loops(g):
        count+=1
count

0

In [37]:
#%timeit sample_pseudo(N,pi)
%timeit sample_bias1(N,pi)
%timeit sample_mask_list(N,pi)
%timeit sample_mask_list(N,pi,"s")
%timeit sample_mask_array(N,pi)
%timeit sample_igraph(N,pi) # seems faster but doesn't seem to generate good graphs

44.3 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.7 ms ± 83.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.7 ms ± 92.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.6 ms ± 77.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
186 µs ± 966 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [75]:
a = np.arange(200).reshape((100,2))  # Example for a
b = np.array([0, 2])    # Example for b, make sure b is shaped appropriately

# Method 1: Using np.any() and np.all() together
exists = np.any(np.all(a == b, axis=1))

print(exists)

False
